The model that incorporated the auxiliary csv data seemed to cause the original CNN to perform worse, so I decided to bring back the original model. This is the final version of the 0.822 VGG19 model w/ Data Augmentation which will hopefully serve as a starting point for the implementation of future models. Everything should be run with GPU.

In [1]:
import numpy as np
import pandas as pd
import os
import gc
import cv2
import tensorflow as tf
import seaborn as sn
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import Model
from keras.utils import plot_model
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
from sklearn.model_selection import train_test_split


print(tf.__version__)

2.2.0


Using TensorFlow backend.


In [2]:
training = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
print(training.head(10))

     image_name  patient_id     sex  age_approx anatom_site_general_challenge  \
0  ISIC_2637011  IP_7279968    male        45.0                     head/neck   
1  ISIC_0015719  IP_3075186  female        45.0               upper extremity   
2  ISIC_0052212  IP_2842074  female        50.0               lower extremity   
3  ISIC_0068279  IP_6890425  female        45.0                     head/neck   
4  ISIC_0074268  IP_8723313  female        55.0               upper extremity   
5  ISIC_0074311  IP_2950485  female        40.0               lower extremity   
6  ISIC_0074542  IP_4698288    male        25.0               lower extremity   
7  ISIC_0075663  IP_6017204  female        35.0                         torso   
8  ISIC_0075914  IP_7622888    male        30.0                         torso   
9  ISIC_0076262  IP_5075533  female        50.0               lower extremity   

  diagnosis benign_malignant  target  
0   unknown           benign       0  
1   unknown           benign  

In [3]:
print(training["target"].value_counts())

0    32542
1      584
Name: target, dtype: int64


We take a very small sample of the benign data to balance out the malignant data.

In [4]:
m = training[training["target"]==1]
b = training[training["target"]==0].sample(2000)
df = pd.concat([m,b])        
df.reset_index(inplace=True)
df.drop(labels=["index", "patient_id", "sex", "age_approx", "anatom_site_general_challenge", "diagnosis", "benign_malignant"], axis=1, inplace=True)
df["image_name"] = "../input/siim-isic-melanoma-classification/jpeg/train/" + df["image_name"].astype(str) + ".jpg"
df.head()

,image_name,target
0,../input/siim-isic-melanoma-classification/jpe...,1
1,../input/siim-isic-melanoma-classification/jpe...,1
2,../input/siim-isic-melanoma-classification/jpe...,1
3,../input/siim-isic-melanoma-classification/jpe...,1
4,../input/siim-isic-melanoma-classification/jpe...,1


Note that we set the random_state in the train-test split as 888. This means the model will be very lucky.

In [5]:
trainX, valX, trainY, valY = train_test_split(
    df["image_name"], 
    df["target"],
    test_size = 0.2, 
    random_state = 888
)
train = list(zip(trainX, trainY))
train = pd.DataFrame(train, columns = ["images", "target"])
val = list(zip(valX, valY))
val = pd.DataFrame(val, columns = ["images", "target"])

train.head()

,images,target
0,../input/siim-isic-melanoma-classification/jpe...,1
1,../input/siim-isic-melanoma-classification/jpe...,0
2,../input/siim-isic-melanoma-classification/jpe...,1
3,../input/siim-isic-melanoma-classification/jpe...,0
4,../input/siim-isic-melanoma-classification/jpe...,0


In [6]:
train_aug = ImageDataGenerator(rescale=1./255,
                     rotation_range=20,
                     width_shift_range=0.2, 
                     height_shift_range=0.2,
                     horizontal_flip=True,
                     vertical_flip=True)

val_aug = ImageDataGenerator(rescale=1./255)

Flow from dataframe reads in images from disk given the filepaths (stored under column "images").

In [7]:
train_gen = train_aug.flow_from_dataframe(train, x_col="images", y_col="target", batch_size = 8, target_size=(224,224),shuffle = True, class_mode="raw")
val_gen = val_aug.flow_from_dataframe(val, x_col="images", y_col="target", batch_size = 8, target_size=(224,224),shuffle = False, class_mode="raw")

Found 2067 validated image filenames.
Found 517 validated image filenames.


In [8]:
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.callbacks import ModelCheckpoint
# early_stop = EarlyStopping(monitor='val_loss', patience=4)
# checkpoint = ModelCheckpoint("{val_loss:.2f}-{epoch:02d}.hdf5",monitor = 'val_loss',verbose = 1,save_best_only = True,mode = 'min')

# callbacks = [early_stop, checkpoint]

In [9]:
from tensorflow.python.keras import backend as K
def focal_loss(alpha=0.25,gamma=2.0):
    def focal_crossentropy(y_true, y_pred):
        bce = K.binary_crossentropy(y_true, y_pred)
        
        y_pred = K.clip(y_pred, K.epsilon(), 1.- K.epsilon())
        p_t = (y_true*y_pred) + ((1-y_true)*(1-y_pred))
        
        alpha_factor = 1
        modulating_factor = 1

        alpha_factor = y_true*alpha + ((1-alpha)*(1-y_true))
        modulating_factor = K.pow((1-p_t), gamma)

        # compute the final loss and return
        return K.mean(alpha_factor*modulating_factor*bce, axis=-1)
    return focal_crossentropy

In [10]:
optimizer = keras.optimizers.Adam(lr=1e-5)
auc = keras.metrics.AUC()

Our classic VGG19 model with no top layer.

In [11]:
vgg = VGG19(include_top=False, weights='imagenet', input_shape = (224,224,3))
flat = Flatten()(vgg.output)
final = Dense(1, activation="sigmoid")(flat)
model = Model(vgg.input,final)
model.summary()

80142336/80134624 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

In [ ]:
model.compile(loss=focal_loss(), metrics=[auc], optimizer=optimizer)

history = model.fit_generator(
    train_gen,
    steps_per_epoch = train.shape[0] // 8,
    epochs = 2, 
    validation_data = val_gen,
    validation_steps = val.shape[0] // 8
)

model.save('vggBASE.h5')

Epoch 1/2
258/258 [==============================] - ETA: 0s - loss: 0.0471 - auc: 0.7242

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch

plt.figure()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.plot(hist['epoch'], hist['loss'], label='Train Error')
plt.plot(hist['epoch'], hist['val_loss'], label='Val Error')
plt.ylim([0, 0.2])
plt.legend()

plt.figure()
plt.xlabel('Epoch')
plt.ylabel('accuracy')
plt.plot(hist['epoch'], hist['auc'], label='Train AUC')
plt.plot(hist['epoch'], hist['val_auc'], label='Val AUC')
plt.ylim([0, 1])
plt.legend()
plt.show()

In [ ]:
del trainX, trainY, valX, valY
gc.collect()

In [ ]:
testing = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
testing.head()

In [ ]:
test_images = np.load('../input/siimisic-melanoma-resized-images/x_test_224.npy')
predictions = []
i=0
for img in tqdm(test_images):       
    img = img.astype(np.float32)/255.
    img=np.reshape(img,(1,224,224,3))       
    pred=model.predict(img)
    predictions.append(pred[0][0])
    i+=1
    if (i==51):
        print(predictions[:50])
    del img
    gc.collect()


In [ ]:
results = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')
results['target'] = predictions
results.to_csv('vgg_predictions11.csv', header=True, index=False)

In [ ]:
results.head()